# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer.

In [ ]:
# ! uv pip install agentics-py

! uv pip install "git+https://github.com/IBM/agentics/@Colab"


import os
import sys

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = input("Enter your GEMINI_API_KEY:")

In [ ]:
from agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional
from agentics.core.llm_connections import get_llm_provider


## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None


class Question(BaseModel):
    question: Optional[str] = None


## Instantiate the source AG with a question
source = AG(
    atype=Question,
    llm=get_llm_provider(),  ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
    ## set verbose to true to see the internal agents log. This is optional.
    states=[Question(question="What is the capital of Italy?")],
)

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target = AG(
    atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
    verbose_agent=True,
)  ## set verbose to true to see the internal agents log

# Execute logical transduction by using the << operator between source and target AG
answer = await (
    target << source
)  ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(
    answer.pretty_print()
)  ## Note that confidence is a float number, while other fields are strings

In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [ ]:
answer = await (AG(atype=Answer) << ["Where is Paris?"])
answer.pretty_print()

## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [ ]:
target = AG(
    atype=Answer,
    verbose_transduction=True,  # Set to verbose to see transduction timings and other logs
    transduction_logs_path="/tmp/answers.jsonl",
)  # Optionally write longs of transductions on the specified path
questions = [
    "Where is Paris?",
    "Who is Alberto Sordi",
    "When will climate change be irreverible?",
    "Who is the best Jeopardy player?",
]

answers = await (target << questions)

answer.pretty_print()

answers = await (target << questions)

## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from pathlib import Path


## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = (
        None  ## Note that fields name should match the column name in the input csv
    )
    genre: Optional[str] = None
    description: Optional[str] = None
    tweet: Optional[str] = Field(
        None, description="Generate a Tweet to advertise the movie"
    )


base = Path(CURRENT_PATH)
movies = AG.from_csv(
    base / "data/movies.csv", atype=Movie, max_rows=20
)  ## Load the input data from a csv file
movies.verbose_transduction = True
movies.llm = get_llm_provider(
    "watsonx"
)  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"

movies_with_tweets = await movies.self_transduction(
    ["movie_name", "genre", "description"],  ## source fields
    ["tweet"],  ## target fields
    ## Note that instruction are only needed when the  relation between source and target type
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie",
)

movies_with_tweets.pretty_print()

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [ ]:
movies = AG.from_csv(base / "data/movies.csv", atype=Movie, max_rows=20)
tweets = await (
    AG(
        atype=movies("tweet").atype,
        instructions="Generate a tweet to advertise the release of the input movie",
    )
    << movies("movie_name", "genre", "description")
)
print(
    tweets.pretty_print()
)  ## Note that differently from self transduction the output tweets
##has only the tweet field, whereas self transduction preserves
## the original source data

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [ ]:
movies = AG.from_csv(base / "data/movies.csv", max_rows=20)
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies):
    print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres = await (movies("genre") << movies("movie_name", "description"))
all_genres.pretty_print()
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)
print(movies_with_genre.pretty_print())